In [1]:
import os
os.chdir('../')
%pwd

'd:\\Python\\Industry level\\EndtoEnd combination\\AI-Resume-Screening-with-ML-ops'

In [2]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DeepModelTrainerConfig:
    root_dir: str
    X_train_path: str
    y_train_path: str
    X_test_path: str
    y_test_path: str
    params: dict


In [3]:
from resumeScreening.constants import *
from resumeScreening.utils.common import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_deep_model_trainer_config(self) -> DeepModelTrainerConfig:
        config = self.config.deep_model_trainer
        create_directories([config.root_dir])

        deep_model_trainer_config = DeepModelTrainerConfig(
            root_dir= config.root_dir,
            X_train_path= config.X_train_path,
            y_train_path= config.y_train_path,
            X_test_path= config.X_test_path,
            y_test_path= config.y_test_path,
            params= self.params.deep_learning
        )

        return deep_model_trainer_config

In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from sklearn.metrics import classification_report, accuracy_score
import pickle
from resumeScreening import logger
import os

In [13]:
class DeepModelTrainer:
    def __init__(self, config):
        self.config = config
        self.X_train = np.load(self.config.X_train_path)
        self.y_train = np.load(self.config.y_train_path)
        self.X_test = np.load(self.config.X_test_path)
        self.y_test = np.load(self.config.y_test_path)

    def build_model(self):
        layers = self.config.params.model.layers
        dropout = self.config.params.model.dropout
        activation = self.config.params.model.activation
        output_activation = self.config.params.model.output_activation

        model = Sequential()
        model.add(Dense(layers[0], activation=activation, input_shape=(self.X_train.shape[1],)))
        model.add(Dropout(dropout))

        for units in layers[1:]:
            model.add(Dense(units, activation=activation))
            model.add(Dropout(dropout))

        model.add(Dense(self.y_train.shape[1], activation=output_activation))

        compile_cfg = self.config.params.compile

        model.compile(
            loss=compile_cfg.loss,
            optimizer=compile_cfg.optimizer,
            metrics=compile_cfg.metrics
        )

        return model

    def train_and_save(self):
        model = self.build_model()

        history = model.fit(
            self.X_train,
            self.y_train,
            epochs=self.config.params.training.epochs,
            batch_size=self.config.params.training.batch_size,
            validation_data=(self.X_test, self.y_test)
        )

        model_path = os.path.join(self.config.root_dir, "deep_model.h5")
        model.save(model_path)
        logger.info(f"Model saved to {model_path}")


In [14]:
try:
    config = ConfigurationManager()
    get_deep_model_trainer_config = config.get_deep_model_trainer_config()
    deep_model_training = DeepModelTrainer(get_deep_model_trainer_config)
    deep_model_training.train_and_save()
except Exception as e:
    raise e

[2025-08-06 05:40:32,977: INFO: yaml file: config\config.yaml loaded successfully]
[2025-08-06 05:40:32,983: INFO: yaml file: params.yaml loaded successfully]
[2025-08-06 05:40:32,988: INFO: Created directory at: artifacts]
[2025-08-06 05:40:32,988: INFO: Created directory at: artifacts/deep_model_trainer]


d:\Python\Industry level\EndtoEnd combination\AI-Resume-Screening-with-ML-ops\resume\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.1951 - loss: 993.6530 - val_accuracy: 0.7876 - val_loss: 47.2807
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6151 - loss: 142.0715 - val_accuracy: 0.9741 - val_loss: 5.6434
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7815 - loss: 52.4073 - val_accuracy: 0.9845 - val_loss: 5.7410
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8531 - loss: 35.8045 - val_accuracy: 0.9845 - val_loss: 6.0637
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8765 - loss: 27.0221 - val_accuracy: 0.9845 - val_loss: 5.8264
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9272 - loss: 13.7191 - val_accuracy: 0.9845 - val_loss: 6.1281
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9363 - loss: 9.7251 - val_accuracy: 0.9845 - val_loss: 5.6530
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9584 - loss: 8.6487 - val_accuracy: 0